<a href="https://colab.research.google.com/github/ShadowRaccoon/PrograConcurrente2023C1/blob/main/TP1_N1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **C++**

In [104]:
%%writefile tp1.cpp
#include <iostream>
#include <unistd.h>
#include <vector>
#include <string>
#include <sys/wait.h>

// Estructura con la que armamos el árbol
typedef struct Node {
  std::string name;
  std::vector<Node> children;
} Node;

int spawnProcesses(Node&);
void processMaessage(Node& n){
    std::cout << "Hola mi nombre es " << n.name << ". PID: " << getpid() << " P_PID: "<< getppid() << std::endl;
}

int main()
{
    Node nodo = {"A", {{"B", {{"E"}, {"F"}}}, {"C"}, {"D", {{"G"}}}}};
    spawnProcesses(nodo);

    return EXIT_SUCCESS;
}


// Función recursiva encargada de crear los procesos hijos y esperarlos
int spawnProcesses(Node& n){
    int pid = 0;
    processMaessage(n);
    for(auto& c : n.children){
        pid = fork();
        if (pid == -1) {
            perror("fork");
            exit(EXIT_FAILURE);
        } else if(pid == 0){
            spawnProcesses(c);
            sleep(50);
            return EXIT_SUCCESS;
        }
    }

    for(const auto& c : n.children){
        wait(nullptr);
    }
    return 0;
}




Overwriting tp1.cpp


In [105]:
!touch resultado_cpp && chmod 700 resultado_cpp

In [106]:
!g++ tp1.cpp -o tp1.out

In [111]:
!nohup ./tp1.out 1>resultado_cpp 2>/dev/null &

In [108]:
!cat resultado_cpp

Hola mi nombre es A. PID: 32259 P_PID: 1
Hola mi nombre es D. PID: 32262 P_PID: 32259
Hola mi nombre es C. PID: 32261 P_PID: 32259
Hola mi nombre es B. PID: 32260 P_PID: 32259
Hola mi nombre es F. PID: 32265 P_PID: 32260
Hola mi nombre es E. PID: 32264 P_PID: 32260
Hola mi nombre es G. PID: 32263 P_PID: 32262


In [112]:
!ps -ef | grep tp1.out

root       33774       1  0 04:16 ?        00:00:00 ./tp1.out
root       33775   33774  0 04:16 ?        00:00:00 ./tp1.out
root       33776   33774  0 04:16 ?        00:00:00 ./tp1.out
root       33777   33774  0 04:16 ?        00:00:00 ./tp1.out
root       33778   33777  0 04:16 ?        00:00:00 ./tp1.out
root       33779   33775  0 04:16 ?        00:00:00 ./tp1.out
root       33780   33775  0 04:16 ?        00:00:00 ./tp1.out
root       33795     273  0 04:16 ?        00:00:00 /bin/bash -c ps -ef | grep tp1.out
root       33797   33795  0 04:16 ?        00:00:00 /bin/bash -c ps -ef | grep tp1.out


In [114]:
!pstree -pc $(pgrep -f tp1.out -P 1)

tp1.out(33774)─┬─tp1.out(33775)─┬─tp1.out(33779)
               │                └─tp1.out(33780)
               ├─tp1.out(33776)
               └─tp1.out(33777)───tp1.out(33778)
